In [2]:
import sys; sys.path.append("../modules")
import random

import numpy as np
import sympy; sympy.init_printing()

import Permutation as pm
from Grid import *

from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
help(pd_to_hfk)

Help on built-in function pd_to_hfk in module knot_floer_homology.hfk:

pd_to_hfk(...)
    >>> pd = 'PD[(5,3,0,2),(1,5,2,4),(3,1,4,0)]'
    >>> HFK = pd_to_hfk(pd)
    >>> sorted(HFK['ranks'].keys())
    [(-1, -2), (0, -1), (1, 0)]
    >>> HFK['total_rank']
    3
    
    >>> pd = 'PD[(2,0,3,19),(6,2,7,1),(4,10,5,9),(15,8,16,9),(13,18,14,19),'
    >>> pd +=   '(7,14,8,15),(10,4,11,3),(17,12,18,13),(0,6,1,5),(11,16,12,17)]'
    >>> HFK = pd_to_hfk(pd)
    >>> HFK['total_rank']
    17
    >>> HFK['tau'], HFK['nu'], HFK['epsilon']
    (0, 0, 0)
    
    >>> pd_code = [(2,0,3,15), (0,6,1,5), (6,2,7,1), (3,11,4,10),
    ...            (11,5,12,4), (7,13,8,12), (13,9,14,8), (9,15,10,14)]
    >>> HFK = pd_to_hfk(pd_code)
    >>> sum(HFK['ranks'].values()) == HFK['total_rank']
    True
    >>> HFK['tau'], HFK['nu'], HFK['epsilon']
    (3, 3, 1)
    
    If the parameter `complex` is set to True, then the simplified
    "UV = 0" knot Floer chain complex is returned. This complex is
    computed

In [3]:
import csv
import ast

knots_data = [] # a list of dicts which has keys 'Name', 'PD Notation', 'Rasmussen s', Ozsvath-Szabo tau', 'L-space', 'Nu' and 'Epsilon'.

with open('knotinfo.csv', 'r') as f:
    dataReader = csv.reader(f)
    keys = next(dataReader)
    for row in dataReader:
        knot_dict = {}
        for col in range(len(row)):
            key = keys[col]
            if key == "PD Notation":
                key = 'PD'
                value = [tuple(l) for l in ast.literal_eval(row[col].translate(str.maketrans({';': ','})))]
            else:
                value = row[col]
            knot_dict[key] = value
        knots_data.append(knot_dict)
            
print(knots_data[1])

{'Name': '13a_2', 'PD': [(3, 1, 4, 26), (1, 8, 2, 9), (7, 2, 8, 3), (9, 5, 10, 4), (5, 14, 6, 15), (13, 6, 14, 7), (15, 11, 16, 10), (11, 19, 12, 18), (19, 13, 20, 12), (23, 17, 24, 16), (17, 23, 18, 22), (25, 20, 26, 21), (21, 24, 22, 25)], 'Rasmussen s': '2', 'Ozsvath-Szabo tau': '1', 'L-space': 'No', 'Nu': '1', 'Epsilon': '1'}


In [4]:
KD = knots_data[999]

#print("[{}] -->  L-space: {}, Nu: {}".format(KD['Name'], KD['L-space'], KD['Nu']))
print({k: v for k,v in KD.items() if k != 'PD'})

pd_to_hfk(KD['PD'])

{'Name': '13a_1000', 'Rasmussen s': '2', 'Ozsvath-Szabo tau': '1', 'L-space': 'No', 'Nu': '1', 'Epsilon': '1'}


{'L_space_knot': False,
 'epsilon': 1,
 'fibered': True,
 'modulus': 2,
 'nu': 1,
 'ranks': {(-4, -5): 1,
  (-3, -4): 10,
  (-2, -3): 40,
  (-1, -2): 86,
  (0, -1): 109,
  (1, 0): 86,
  (2, 1): 40,
  (3, 2): 10,
  (4, 3): 1},
 'seifert_genus': 4,
 'tau': 1,
 'total_rank': 383}

In [5]:
for kd in tqdm(knots_data):
    knotinfo_ep = ast.literal_eval(kd['Ozsvath-Szabo tau'])
    result = pd_to_hfk(kd['PD'])
    calculated_ep = result['tau']
    if knotinfo_ep != calculated_ep:
        #print("Different!!")
        print(kd)
        print(result)
        exit

  0%|          | 0/9988 [00:00<?, ?it/s]

In [ ]:
import os
import multiprocessing

MAX_COUNT = 100000000
ITERATION = 50000000

def whoami(what):
    #単純な加算
    count = 0
    for n in range(MAX_COUNT):
        if count % ITERATION ==0:
            #実行中のプロセスIDと,現在のcount数を表示
            print("{} Process {} count {}".format(what,os.getpid(),count))
        count +=1
    #どのIDのプロセスが終了したかを表示
    print("end {} Process {}".format(what,os.getpid()))
#現在のプロセスのidを表示
print("Main Process ID is {}".format(os.getpid()))
#メインのプロセスで実行
whoami("main program")

print("-----------------------------------------------------")
#プロセスを10作りスタートさせる.
for n in range(10):
    p = multiprocessing.Process(target=whoami,args=("Process {}".format(n),))
    p.start()
print("end of program")

